In [5]:
# Example for classifying forams in paleo oceanography Kaggle competition.
# Author: Peter Sadowski
# Date: Nov 2 2024
# This runs on google colab TPU, and achieves 57% accuracy on public leaderboard.

In [6]:
# # Download data files from public google drive.
# # Takes ~1min
# !pip install --upgrade --no-cache-dir gdown
# import gdown
# 
# file_ids = {'train.zip': '1mm2tEB05wQwNHP0SySTCp1-BL6G1IHf0',
#             'test.zip': '1Cf-yAfSt706w10p5Dij7ppFLo6Se8Ej7',
#             'train.csv': '1NAd3UPTWWxmXdJ9N_Mr3GWgMmO_Aojnn',
#             'test.csv': '1Koi9hpUuUwn1swel9QLsEOr_MZKX2LOg',
#             }
# 
# for output, file_id in file_ids.items():
#   url=f"https://drive.google.com/uc?id={file_id}"
#   gdown.download(url, output, quiet=False)
# 
# !unzip -q train.zip
# !unzip -q test.zip


In [7]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

# https://stackoverflow.com/questions/72371859/attributeerror-module-collections-has-no-attribute-iterable
import collections
collections.Iterable = collections.abc.Iterable

class ImageCSVDataset(Dataset):
    def __init__(self, image_dir, csv_file, transform=None, test_set=False):
        """
        Args:
            image_dir (string): Directory with all the images.
            csv_file (string): Path to the csv file with labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.

        """
        self.image_dir = image_dir
        self.labels = pd.read_csv(csv_file)
        self.transform = transform
        self.test_set = test_set

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.labels.iloc[idx]['filename']  # Assuming image name is in the first column
        image = Image.open(f"{self.image_dir}/{img_name}").convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.test_set:
          return image

        label = self.labels.iloc[idx]['label']  # Assuming label is in the second column
        return image, label

# Define dataset and data loader.
image_dir = 'train/'
csv_file = 'train.csv'
image_dir_test = 'test/'
csv_file_test = 'test.csv'

# Define transformations (optional)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),          # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.225, 0.225, 0.225])  # Normalize
])

dataset = ImageCSVDataset(image_dir=image_dir, csv_file=csv_file, transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [8]:
# Train a NN from scratch.
import torch
import torch.nn as nn
import torch.optim as optim

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 128),  # Adjust input size based on image dimensions
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Instantiate the model, optimizer, and loss function
#num_classes = len(dataset.labels['label'].unique())  # Get number of unique classes from the dataset
num_classes = dataset.labels['label'].max()+1  # Get number of unique classes from the dataset
model = SimpleCNN(num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")

/mnt/dev/PycharmProjects/ics637/venv/lib/python3.13/site-packages/torchvision/transforms/functional.py:63: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


[1, 100] loss: 2.180
[1, 200] loss: 1.521
Finished Training


In [10]:
# Use model to make predictions.

# Create test dataloader that doesn't have labels.
dataset_test = ImageCSVDataset(image_dir=image_dir_test, csv_file=csv_file_test,
                               transform=transform, test_set=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

predictions = []
for i, data in enumerate(dataloader_test, 0):
    inputs = data.to(device)
    outputs = model(inputs)
    predictions.append(outputs.argmax(axis=1).detach().cpu().numpy())
predictions = np.concatenate(predictions)

# Write predictions to a submission file.
df_predictions = pd.read_csv(csv_file_test)
df_predictions['predictions'] = predictions
# df_predictions[['id', 'predictions']].to_csv('submission.csv', index=False)

display(df_predictions)


In [12]:
# Fine tune a pre-trained model
from torchvision import models

# Load a pre-trained model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Replace with your number of classes

# Freeze early layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Unfreeze later layers (optional)
for layer in list(model.children())[-3:]:  # Unfreeze last 3 layers
    for param in layer.parameters():
        param.requires_grad = True

# 4. Define optimizer and loss function
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 5. Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 6. Train the model (similar to previous example)
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/echung32/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 30.9MB/s]


[1, 100] loss: 1.032
[1, 200] loss: 0.786
[2, 100] loss: 0.549
[2, 200] loss: 0.552
[3, 100] loss: 0.402
[3, 200] loss: 0.388
[4, 100] loss: 0.245
[4, 200] loss: 0.278
[5, 100] loss: 0.178
[5, 200] loss: 0.200
[6, 100] loss: 0.115
[6, 200] loss: 0.110
[7, 100] loss: 0.088
[7, 200] loss: 0.098
[8, 100] loss: 0.060
[8, 200] loss: 0.058
[9, 100] loss: 0.042
[9, 200] loss: 0.061
[10, 100] loss: 0.039
[10, 200] loss: 0.055


In [16]:
# Use model to make predictions.

# Create test dataloader that doesn't have labels.
dataset_test = ImageCSVDataset(image_dir=image_dir_test, csv_file=csv_file_test,
                               transform=transform, test_set=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

predictions = []
for i, data in enumerate(dataloader_test, 0):
    inputs = data.to(device)
    outputs = model(inputs)
    predictions.append(outputs.argmax(axis=1).detach().cpu().numpy())
predictions = np.concatenate(predictions)

# Write predictions to a submission file.
df_predictions = pd.read_csv(csv_file_test)
df_predictions['predictions'] = predictions
# df_predictions[['id', 'predictions']].to_csv('submission.csv', index=False)

display(df_predictions)

,id,filename,predictions
0,0,MV1012-BC-12_obj00007.jpg,1
1,1,MV1012-BC-12_obj00009.jpg,3
2,2,MV1012-BC-12_obj00011.jpg,1
3,3,MV1012-BC-12_obj00012.jpg,3
4,4,MV1012-BC-12_obj00013.jpg,5
...,...,...,...
2169,2169,MV1012-BC-8_obj01887.jpg,0
2170,2170,MV1012-BC-8_obj01892.jpg,0
2171,2171,MV1012-BC-8_obj01893.jpg,0
2172,2172,MV1012-BC-8_obj01899.jpg,1
